# Scraping Metal Archives et Sens Critique 

# Scrapping avec Sélénium

## Metal-Archives.com

In [21]:
#Import des librairies nécessaires 

from bs4 import BeautifulSoup
import requests 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
import time
import pandas as pd
import re
import random as rd

## Récupération de la liste de groupes par scraping sur le site spirit-of-metal.com

In [22]:
'''
Fonction permettant de scroller à l'infini sur la page du site contenant la liste des groupes contenant les noms
des groupes que l'on souhaite récupérer. 

'''

def scroll(driver, timeout):
    scroll_pause_time = timeout
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        
        time.sleep(scroll_pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # If heights are the same it will exit the function
            break
        last_height = new_height


In [8]:
'''
Code permettant d'accéder à la page contenant les noms, de scroller à l'infini et de récupérer par la suite
de récuperer les noms de groupe dans une liste. On utilise ensuite ensuite une Regex pour éliminer les parenthèses si besoin
pour récupérer des noms propres. A la fin, on affiche la longueur de la liste qui permet de donner le nombre
potentiel de groupes que l'on peut scraper. Pour le projet, on ne scrappera qu'une partie des groupes. 

'''
driver = webdriver.Firefox()
driver.get("https://www.spirit-of-metal.com/liste_groupe.php?recherche_groupe=&lettre=&id_pays_recherche=0&id_style_recherche=0&dateCrea=0&nb_etoile=2")
scroll(driver,2)

time.sleep(3)




In [9]:
groupes=driver.find_elements_by_class_name("col-xs-9")
bands=[]
for e in groupes:
    e=e.text
    e=e.split("\n")[0]
    cleanr = re.compile('(\((.*)\))')
    e= re.sub(cleanr, '',e)
    bands.append(e)
print(len(bands))

1976


In [37]:
bands

['12 Stones',
 '12012',
 '1349',
 '25 Ta Life',
 '3 Inches Of Blood',
 '30 Seconds To Mars',
 '36 Crazyfists',
 'A Day To Remember',
 'A Different Breed Of Killer',
 'A Forest Of Stars',
 'A Perfect Circle',
 'A Plea For Purging',
 'A Skylit Drive',
 'A Static Lullaby',
 'A Vain Attempt',
 'A9',
 'Aäkon Këëtrëh',
 'Aaskereia',
 'Abacabb',
 'Abandon All Ships',
 'Abbath',
 'Abigail ',
 'Abigail Williams',
 'Abigor',
 'Abingdon Boys School',
 'Abnormality ',
 'Abominable Putridity',
 'Aborted',
 'Aborym',
 'Abruptum',
 'Absu',
 'Abysmal Dawn',
 'Abysmal Torment',
 'Abyssic Hate',
 'Abyssos',
 'AC-DC',
 'Accept',
 'Accuser',
 'Ace Frehley',
 'Acheron ',
 'Acherontas',
 'Acid Bath',
 'Acid Drinkers',
 'Acid King',
 'Acid Witch',
 'Acrania ',
 'Ad Hominem',
 'Adagio ',
 'Adelitas Way',
 'Adema',
 'Adept',
 'Adorned Brood',
 'Adrenaline Mob',
 'Adrian Von Ziegler',
 'ADX',
 'Aeon ',
 'Aerosmith',
 'Aes Dana',
 'Aesma Daeva',
 'Aeternam',
 'Aeternus',
 'AFI',
 'After Forever',
 'After The Bur

### Code du scraping sur Spirit-of-Metal.com 

Ce code consiste à aller sur l'URL du groupe recherché en plaçant le nom du groupe à la fin de l'URL, d'aller sur le tableau contenant toutes les infos du groupe sur la page puis enfin de récupérer les informations recherchées: nom, 
genre, statut (groupe toujours actif ou non), date de formation et pays d'origine. 

Au cas où le code renvoie une erreur indiquant que les éléments ne peuvent pas être trouvées, on passe alors par le moteur de recherche du groupe, on entre le nom du groupe recherché, on récupère le lien qui permet d'accéder à la page du groupe. On accède à la page et on récupère par la même méthode les informations recherchées. 

In [23]:
from selenium.common.exceptions import *
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
def scrapping_spirit(band):
    try:
        driver = webdriver.Firefox()
        driver.get("https://www.spirit-of-metal.com/fr/band/{}".format(band))
        elements=driver.find_element_by_id("profile")
        element=elements.find_elements_by_tag_name("span")
        time.sleep(3)
        band_infos=[i.text for i in element]
        i=band_infos.index("Genre")
        genre=band_infos[i+1]
        i=band_infos.index("Statut")
        statut=band_infos[i+1]
        i=band_infos.index("Date de formation")
        annee=band_infos[i+1]
        i=band_infos.index("Pays")
        pays=band_infos[i+1]
        time.sleep(1.5)
        return band,pays,genre,statut,annee
    except NoSuchElementException:
        print("première méthode échouée. Nouvelle Tentative !!")
        driver=webdriver.Firefox()
        driver.get("https://www.spirit-of-metal.com/fr/bands/1")
        element=driver.find_elements_by_name("recherche_groupe")[1]
        element.clear()
        element.send_keys(band)
        driver.execute_script("window.scrollTo(1023,713);")
        recherche=driver.find_element_by_xpath("/html/body/main/div/div/aside/section[1]/div/form/input[3]")
        recherche.click()
        #driver.send_keys(Keys.ENTER)
        groupe=driver.find_elements_by_xpath("/html/body/main/div/div/div/section[3]/div[1]/div/a")
        lien=groupe[0].get_attribute("href")
        driver.get(lien)
        elements=driver.find_element_by_id("profile")
        element=elements.find_elements_by_tag_name("span")
        time.sleep(2)
        band_infos=[i.text for i in element]
        i=band_infos.index("Genre")
        genre=band_infos[i+1]
        i=band_infos.index("Statut")
        statut=band_infos[i+1]
        i=band_infos.index("Date de formation")
        annee=band_infos[i+1]
        i=band_infos.index("Pays")
        pays=band_infos[i+1]
        time.sleep(1.5)
        driver.quit()
        return band,pays,genre,statut,annee

## Code pour le scrapping sur le site Métal-Archives. 

On accède d'abord à la page du moteur de recherche avancée du site, on entre le nom du groupe recherché dans le champ consacré au nom. En cas de bugs rencontrées, on recharge la page et on recommence les étapes précédentes. 

Par la suite, on clique sur le bouton pour lancer la recherche. Puis, une fois les résultats générés, on récupère le lien du premier groupe de la barre de recherche. On accède à la page du groupe et dans le tableau en haut de la page, on récupère les informations recherchées : nom, genre, statut, pays, années d'activité, thèmes des paroles. Si la recherche ne retoune rien ou une erreur est rencontrée lors de la chargement de la page, on indique que le groupe n'est pas présent dans la base de données du site et on renvoie le nom du groupe. 

Néanmoins, j'ai constaté un problème difficile à surmonter lors du scraping sur une grande liste de groupes : la présence d'homonyms au sein du groupe au sein du site. Par conséquent, ce n'est pas forcément les infos du groupe recherché en premier lieu qui seront recupérées, mais celles d'un homonyme ou d'une formation dont le nom matche lors de la recherche. Pour pallier au problème, si l'on souhaite un groupe en particulier, il conviendra dans ce cas de récupérer l'URL du groupe recherché et de scraper à partir de celle-ci. 

In [24]:

def scrapping_groupes(band):
    try:
        driver = webdriver.Firefox()
        driver.get("https://www.metal-archives.com/search/advanced/?searchString=&type=")
        time.sleep(2)
        try:
            groupe2=driver.find_element_by_xpath("""//*[@id="bandName"]""")
            groupe2.clear()
            groupe2.send_keys(band)
        except (ElementNotInteractableException, StaleElementReferenceException):
            print("erreur sur la page. Page rafraîchie!!!")
            driver.refresh()
            time.sleep(3)
            #driver.execute_script("arguments[0].scrollIntoView();", groupe2)
            groupe2=driver.find_element_by_xpath("""//*[@id="bandName"]""")
            groupe2.clear()
            groupe2.send_keys(band)
            
            pass
            
            
       
        print(band)
        #genre=driver.find_element_by_id("genre")
        #genre.clear()
        #g=input("genre:")
        #genre.send_keys(g)
        bouton=driver.find_element_by_class_name("defaultButton")
        bouton.click()
        time.sleep(1)
        a=driver.find_element_by_xpath("/html/body/div/div[3]/div/div[1]/div/div/div/div/div[5]/table/tbody/tr[1]/td[1]/a").get_attribute("href")


        driver.get(a)
        print(driver.current_url)
        statut=driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div[2]/div[2]/dl[1]/dd[3]").text
        nom=driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div[2]/h1/a").text
        pays=driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div[2]/div[2]/dl[1]/dd[1]/a").text
        annees=driver.find_element_by_xpath("/html/body/div/div[3]/div[2]/div[2]/div[2]/dl[3]/dd").text
        annees= re.sub("</strong>|<dd>|</dd>", "",annees)
        cleanr = re.compile('<.*>|[(a-zA-Z)]')
        annees= re.sub(cleanr, '',annees)
        genre=driver.find_element_by_css_selector("dl.float_right > dd:nth-child(2)").text
        lyrics=driver.find_element_by_css_selector("dl.float_right > dd:nth-child(4)").text
        time.sleep(2)
        driver.quit()
        
    except (NoSuchElementException,AttributeError,TypeError,IndexError):
        print(band," non présent dans Métal Archives")
        band3=band
        return band3
    return nom,pays,statut,genre,annees,lyrics




# Choix dans la liste "bands" des groupes que l'on souhaite scraper. 

En effet, pour des raisons pratiques pour pouvoir remédier aux bugs arrivant fréquemment durant le scraping, on lance un processus de scraping pour une cinquantaine de groupes à la fois. On se base pour cela sur l'index des groupes recherchés, autrement dit, leur position dans la liste. Pour les cinq cents premiers, le scraping se déroule selon l'ordre dans lequel les groupes ont été récupérés donc par ordre alphabétique. 

Par contre, pour une plus grande diversité dans les noms des groupes dans le dataset final du projet, par la suite, on scrappera toujours de la même façon, mais la liste utilisée sera mélangée aléatoirement. Un mélange qui sera maintenu par la suite, peu importe le nombre de fois que le code sera relancé de façon à pouvoir travailler avec la même liste à chaque fois que le code sera relancé. Néanmoins, au vu du mélange qui a tendance à ne pas être touours identique après chaque relance de Jupyter, on veillera à aller jusqu'au bout de chaque processus de scrapping pour éviter les soucis.

In [54]:
#Récupération des groupes à partir du 500ème. 
gps=bands[500:]


In [55]:
'''
Mélange aléatoire de la liste en fixant un random seed. Ce paramètre permet de maintenir le mélange si le code
est relancé plusieurs fois. 
'''

import random as rd
rd.seed(4)
rd.shuffle(gps)

In [58]:
'''
Dans cette case, on définit trois listes à la suite pour déterminer la liste des groupes à récupérer
pour chaque processusde scrapping. Une fois le processus de scrapping terminé pour chaque liste, 
on change les indices dans les trois listes pour poursuivre le scrapping.

'''

groupes17=gps[300:350]
groupes18=gps[350:400]
groupes19=gps[400:450]
groupes19

['From Autumn To Ashes',
 'Mortal Love',
 'The Black Dahlia Murder',
 'Gojira',
 'Saxon',
 'Get Scared',
 'Woods Of Ypres',
 'Kittie',
 'ReVamp',
 'Vulcain',
 'Pitbulls In The Nursery',
 'Puddle Of Mudd',
 'Noctem',
 'Thy Art Is Murder',
 'Sick Of It All',
 'Panzerchrist',
 "Grand Belial's Key",
 'Falconer',
 'Lord Belial',
 'Estatic Fear',
 'Papa Roach',
 "King's X",
 'Hate Forest',
 'Moi Dix Mois',
 'Rev Theory',
 'Firehouse ',
 'Marilyn Manson',
 'King Of Asgard',
 'Silent Civilian',
 'Stream Of Passion',
 'Pyramaze',
 'Ultra Vomit',
 'Man Must Die',
 'Ruoska',
 'Vildhjarta',
 'In The Midst Of Lions',
 'Massacra',
 'Mystifier',
 'Flyleaf',
 'Necrowretch',
 'The Acacia Strain',
 'Tengger Cavalry',
 'Revenge ',
 'Undercover Slut',
 'Lacrimosa',
 'Old Funeral',
 'Megadeth',
 'Vomitory ',
 'The Devil Wears Prada',
 'Funeral Mist']

## Code du scrapping  à proprement parler pour une liste de 50 groupes définie précédemment. 

Pour chaque groupe, on scrappe d'abord sur Metal-Archives. Si la recherche ne contient pas le groupe recherché, on renvoie le nom du groupe et on passe au scrapping sur Spirit-of-Metal. Les informations récupérées lors du scraping sont affichées à chaque fois pour vérifier qu'elles sont bien récupérées et sont stockées dans différentes listes. On instancie  également un compteur qui permet de suivre l'avancement du scraping en temps réel en comptant le nombre de groupes déjà scrappés au fur et à mesure. Dans ce code, on a ajouté également au fur et à mesure la gestion des différentes erreurs rencontrées lors du scraping. Néanmoins, le scraping n'est pas exempt de bugs. Par conséquent, en cas d'erreurs rencontrées lors du scraping, il conviendra désormais de relancer la cellule. 

Par la suite, les différentes listes sont intégrées à un dictionnaire vide, puis un DataFrame est créé à partir de celui-ci. Enfin, on sauve le DataFrame ainsi créé au format CSV de façon à pouvoir sauvegarder les données au fur et à mesure pour ne rien perdre. 



In [30]:

noms=[]
pays=[]
statuts=[]
genres=[]
annees=[]
paroles=[]
c=1
for elem in groupes18:
    print(elem)
    infos=scrapping_groupes(elem)
    print(infos)
    if infos!=elem:
        noms.append(infos[0])
        pays.append(infos[1])
        statuts.append(infos[2])
        genres.append(infos[3])
        annees.append(infos[4])
        try:
            paroles.append(infos[5])
        except IndexError:
            continue
    else:
        print("méthode spirit")
        infos2=scrapping_spirit(elem)
        print(infos2)        
        noms.append(infos2[0])
        pays.append(infos2[1])
        genres.append(infos2[2])
        statuts.append(infos2[3])
        annees.append(infos2[4])
        paroles.append("NA")
    print(c,"/",len(groupes18),"groupes scrappés")
    c+=1

    

Poison The Well
Poison The Well
Poison The Well  non présent dans Métal Archives
Poison The Well
méthode spirit
('Poison The Well', 'Etats-Unis', 'Post Hardcore', 'Split-Up', '1997')
1 / 50 groupes scrappés
Tony MacAlpine
Tony MacAlpine
https://www.metal-archives.com/bands/Tony_MacAlpine/3540333427
('Tony MacAlpine', 'United States', 'Active', 'Shred, Neoclassical, Progressive Rock/Metal', '1985-', 'Instrumental')
2 / 50 groupes scrappés
High On Fire
High On Fire
https://www.metal-archives.com/bands/High_on_Fire/2931
('High on Fire', 'United States', 'Active', 'Stoner/Sludge/Doom Metal', '1998-', 'Mythologies, Drugs')
3 / 50 groupes scrappés
Gallhammer
Gallhammer
https://www.metal-archives.com/bands/Gallhammer/28291
('Gallhammer', 'Japan', 'On hold', 'Black/Doom Metal, Crust', '2003-2013', 'Delirium, Paranoia, Hatred, Despair')
4 / 50 groupes scrappés
Underoath
Underoath
https://www.metal-archives.com/bands/Underoath/98854
('Underoath', 'United States', 'Active', 'Metalcore (early), Al

('Gary Moore', 'Royaume-Uni', 'Guitar Hero', 'Split-Up', '1973')
39 / 50 groupes scrappés
Nonpoint
Nonpoint
https://www.metal-archives.com/bands/Nonpoint_Factor/51586
('Nonpoint Factor', 'Puerto Rico', 'Changed name', 'Thrash Metal', '1991-1996, 1996-  ', 'Personal struggles')
40 / 50 groupes scrappés
Immortal
Immortal
https://www.metal-archives.com/bands/Immortal/25791
('Immortal', 'Denmark', 'Split-up', 'Death Metal', '1988-?', 'Reincarnation')
41 / 50 groupes scrappés
Night Ranger
Night Ranger
Night Ranger  non présent dans Métal Archives
Night Ranger
méthode spirit
('Night Ranger', 'Etats-Unis', 'Hard FM', 'Actif', '1982')
42 / 50 groupes scrappés
Still Remains
Still Remains
https://www.metal-archives.com/bands/Still_Remains/3540368801
('Still Remains', 'United Kingdom', 'Active', 'Heavy Metal/Hard Rock', '1993-', 'N/A')
43 / 50 groupes scrappés
Woe, Is Me
Woe, Is Me
https://www.metal-archives.com/bands/Woe_unto_Me/3540375442
('Woe unto Me', 'Belarus', 'Active', 'Funeral Doom Metal

In [31]:
res={}
res['nom']=noms
res['pays']=pays
res['statut']=statuts
res['genre']=genres
res['years']=annees
res['lyrical themes']=paroles
df=pd.DataFrame.from_dict(res,orient="columns")
df

,nom,pays,statut,genre,years,lyrical themes
0,Poison The Well,Etats-Unis,Split-Up,Post Hardcore,1997,NA
1,Tony MacAlpine,United States,Active,"Shred, Neoclassical, Progressive Rock/Metal",1985-,Instrumental
2,High on Fire,United States,Active,Stoner/Sludge/Doom Metal,1998-,"Mythologies, Drugs"
3,Gallhammer,Japan,On hold,"Black/Doom Metal, Crust",2003-2013,"Delirium, Paranoia, Hatred, Despair"
4,Underoath,United States,Active,"Metalcore (early), Alternative Rock/Post-Hardc...","1997-2013, 2015-","Christianity (early); Consequence, Heartbreak,..."
5,"Drop Dead, Gorgeous",Etats-Unis,Split-Up,Metalcore,2004,NA
6,Grand Magus,Sweden,Active,Heavy/Doom Metal,"1996-1999 , 1999-","Paganism, Anti-Christianity, Strength, Death"
7,Possessed,United States,Active,Death/Thrash Metal,"1983-1987, 1990-1993, 2007-","Satanism, Occultism, Anti-Christianity, Murder..."
8,Wolfmother,Australie,Actif,Stoner,2000,NA
9,Hinder,France,Changed name,Progressive Dark Metal,"2003-2004 ä, 2004-?, ?-",N/A


In [32]:
df.to_csv('df18.csv', index=False)

## Récupération des votes des utilisateurs et des groupes placés en favoris à partir du site Sens-Critique.com 


Une fois les infos de groupes récupérées, on scrappe le site Sens-Critique.com pour récupérer deux informations intéressantes pour enrichir la recommandation: la note moyenne des albums de chaque groupe ainsi que le nombre de fois que le groupe a été ajouté comme groupe favori par l'utilisateur. Ces informations permettent de juger de la qualité musicale d'un groupe ainsi que de sa popularité auprès des utilisateurs. 

On utilise une méthode similaire que dans le premier cas : on va sur le moteur de recherche du site en sélectionnant une recherche par nom, on lance la recherche, on récupèrer le lien du premier groupe dans les résultats et on se rend sur la page du groupe en question. On récupère toutes les notes des albums présents dans la page. On les convertit ensuite en données chiffrées afin d'en calculer la moyenne. On récupère également le nombre de favoris (autrement dit, le nombre situées à côté du coeur rouge sous le nom du groupe tout en haut de la page). Une fois les informations récupérées, on renvoie le nom du groupe, la moyenne des notes et le nombre de favoris. 

On constate d'aileurs ici le même problème rencontré lors du scraping des informations de groupe : la présence d'homonymes au sein des résultats de recherche. Pour régler ce problème, il conviendra de lister les groupes avec plus de 100 favoris dans le DataFrame final. En effet, il est rare qu'un groupe de métal sur Sens-Critique ait plus de 100 favoris. 
On vérifiera ensuite directement sur le site pour vérifier si la page atteinte par le scrapping est bien celle du groupe recherché. Sinon, on supprimera l'entrée du groupe en question et on relancera le scrapping directement à partir de l'URL correspondant au groupe recherché. 


In [19]:
def scrapping_sens_critique(elem):
    note=[]
    likes=driver.find_element_by_class_name("cvi-totallike").get_attribute("data-sc-like-count")
    c=driver.find_elements_by_class_name("erra-ratings")
    for i in c:
        b=i.find_element_by_tag_name("a")
        if b.text!='-':
            note.append(float(b.text))
    moy=round(np.mean(note),2)
    time.sleep(2)
    driver.quit()
    return elem,moy,likes



In [18]:
'''
Code alternatif permettant de relancer la procédure de scraping au cas où la première méthode ne fonctionne pas. 
Autrement dit, si le scrapping tombe sur une page de recherche ne contenant aucun groupe. Dans ce cas, on utilise
la même procédure, mais en utilisant une méthode différente pour récupérer l'URL du groupe recherché. 
'''

def alternative_spirit(elem):
    notes=[]
    driver = webdriver.Firefox()
    driver.get("https://www.senscritique.com/searchArtist?q={}".format(elem))
    elem=re.findall("[a-zA-Z]+",elem)
    elem=str(",".join(elem)).replace(","," ")
    a=driver.find_elements_by_css_selector("html body div#__next div.search__StyledGrid-sc-1mr42vn-0.deACLc div.search__StyledResultsColumn-sc-1mr42vn-4.igWMhV div.sk-hits.SearchArtist__HitsStyled-soe2rz-0.iFPMuN div.SearchArtistResult__Container-sc-19zp711-0.eACiIN a.SearchArtistResult__Title-sc-19zp711-5.bxizrx")
    lien=[i.get_attribute("href") for i in a]
    a1=lien[1]
    a1=a1+"#page-1/order-note/"
    driver.get(a1)
    favoris=driver.find_element_by_class_name("cvi-totallike").get_attribute("data-sc-like-count")
    b=driver.find_elements_by_class_name("erra-global")
    note=[]
    c=driver.find_elements_by_class_name("erra-ratings")
    for i in c:
        b=i.find_element_by_tag_name("a")
        if b.text!='-':
            note.append(float(b.text))
    moy=round(np.mean(notes),2)
    time.sleep(2)
    driver.quit()
    return elem,moy,favoris


In [33]:
'''
Détermination des listes pour chaque processus de récupération des données. Les noms sont différents, 
mais on fait coincider les indices de façon à obtenir les mêmes listes de  groupes à scrapper 
que lors du scrapping des informations sur les groupes. 

'''

bandes17=gps[300:350]
bandes18=gps[350:400]
bandes19=gps[400:450]
bandes18


['Poison The Well',
 'Tony MacAlpine',
 'High On Fire',
 'Gallhammer',
 'Underoath',
 'Drop Dead, Gorgeous',
 'Grand Magus',
 'Possessed',
 'Wolfmother',
 'Hinder ',
 'Teräsbetoni',
 'Revocation',
 'Enter Shikari',
 'Master ',
 'Nebelhorn',
 'Deicide',
 'Thunder ',
 'Inhumate',
 'Minas Morgul ',
 'Get Scared',
 'Rotten Sound',
 'Survivor',
 'Dodheimsgard',
 'X Japan',
 'Tierra Santa',
 'Mors Principium Est',
 'Suffocation ',
 'Iron Maiden ',
 'Graveland',
 'Oomph',
 'Doug Wimbish',
 'Zakk Wylde',
 'Nunslaughter',
 'Lita Ford',
 'Earth Crisis',
 'Molly Hatchet',
 'Icon For Hire',
 'I Set My Friends On Fire',
 'Gary Moore',
 'Nonpoint',
 'Immortal',
 'Night Ranger',
 'Still Remains',
 'Woe, Is Me',
 'Massacra',
 'Infernäl Mäjesty',
 'Hellveto',
 'Stryper',
 'Forest Silence',
 'From First To Last']

## Code du scrapping à proprement parler avec les deux méthodes contenues dans les fonctions précédentes pour chacune des listes définies précedemment. 

In [34]:
c=1
favoris=[]
moyennes=[]
groupes=[]
scrap2=[]
for elem in bandes18:
    try:
        driver = webdriver.Firefox()
        driver.get("https://www.senscritique.com/searchArtist?q={}".format(elem))
        a=driver.find_element_by_xpath("/html/body/div[1]/div[2]/div[3]/div[2]/div[1]/a").get_attribute("href")
        a=a+"#page-1/order-note/"
    except NoSuchElementException:
        print(elem," non trouvé.")
        scrap2.append(elem)
        print(elem," à rescrapper !")
    
    driver.get(a)
    if driver.current_url=="https://www.senscritique.com/#page-1/order-note/":
        driver.quit()
        alt=alternative_spirit(elem)
        favoris.append(alt[2])
        moyennes.append(alt[1])
        groupes.append(alt[0])
        print(alt[0],alt[1],alt[2])
    
        
    else:
        result=scrapping_sens_critique(elem)
        favoris.append(result[2])
        moyennes.append(result[1])
        groupes.append(result[0])
        print(result[0],result[1],result[2]) 
    print(c,"sur",len(bandes18),"groupes scrappés")
    c+=1


Poison The Well 7.17 4
1 sur 50 groupes scrappés
Tony MacAlpine 7.47 4
2 sur 50 groupes scrappés
High On Fire 7.1 12
3 sur 50 groupes scrappés
Gallhammer 7.43 7
4 sur 50 groupes scrappés
Underoath 6.58 15
5 sur 50 groupes scrappés


C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Drop Dead, Gorgeous nan 0
6 sur 50 groupes scrappés
Grand Magus 6.92 12
7 sur 50 groupes scrappés
Possessed 7.15 7
8 sur 50 groupes scrappés
Wolfmother 7.0 69
9 sur 50 groupes scrappés
Hinder  nan 2
10 sur 50 groupes scrappés
Teräsbetoni nan 3
11 sur 50 groupes scrappés
Revocation 7.22 17
12 sur 50 groupes scrappés
Enter Shikari 7.56 68
13 sur 50 groupes scrappés
Master  7.9 20
14 sur 50 groupes scrappés
Nebelhorn nan 0
15 sur 50 groupes scrappés
Deicide 6.57 34
16 sur 50 groupes scrappés
Thunder  6.55 11
17 sur 50 groupes scrappés
Inhumate 6.7 0
18 sur 50 groupes scrappés
Minas Morgul  nan 0
19 sur 50 groupes scrappés
Get Scared 6.53 4
20 sur 50 groupes scrappés
Rotten Sound 7.45 7
21 sur 50 groupes scrappés
Survivor 7.01 17
22 sur 50 groupes scrappés
Dodheimsgard 7.43 12
23 sur 50 groupes scrappés
X Japan 8.15 88
24 sur 50 groupes scrappés
Tierra Santa 6.75 2
25 sur 50 groupes scrappés
Mors Principium Est 7.25 19
26 sur 50 groupes scrappés
Suffocation  7.23 25
27 sur 50 groupes scrap

In [35]:
# On construit le dataset de la même façon que l'étape précédente. 


import pandas as pd
ratings={}
ratings["nom"]=groupes
ratings["note moyenne"]=moyennes
ratings["favoris"]=favoris
dfr=pd.DataFrame.from_dict(ratings,orient='columns')
dfr

,nom,note moyenne,favoris
0,Poison The Well,7.17,4
1,Tony MacAlpine,7.47,4
2,High On Fire,7.10,12
3,Gallhammer,7.43,7
4,Underoath,6.58,15
5,"Drop Dead, Gorgeous",NaN,0
6,Grand Magus,6.92,12
7,Possessed,7.15,7
8,Wolfmother,7.00,69
9,Hinder,NaN,2


In [37]:
'''
Petit preprocessing pour remplacer les valeurs nulles par la valeur 0 et supprimer les doublons 
à l'issue du scrapping. 
'''

dfr["note moyenne"]=dfr["note moyenne"].fillna(0)
dfr=dfr.drop_duplicates()
dfr

,nom,note moyenne,favoris
0,Poison The Well,7.17,4
1,Tony MacAlpine,7.47,4
2,High On Fire,7.10,12
3,Gallhammer,7.43,7
4,Underoath,6.58,15
5,"Drop Dead, Gorgeous",0.00,0
6,Grand Magus,6.92,12
7,Possessed,7.15,7
8,Wolfmother,7.00,69
9,Hinder,0.00,2


In [38]:
#Sauvegarde du dataset obtenu
dfr.to_csv('ratings18.csv', index=False)

# Obtention du dataset final par fusion des Dataframes issues des deux scrappings sur la colonne "nom". 

On constate au passage que certains groupes sont perdus si les noms utilisées dans les deux Dataframes ne matchent pas parfaitement. Dans ce cas, il conviendra de rescrapper individuellement les groupes perdus. On utilisera si besoin la méthode "read_csv" pour merger si l'on a sauvegardé les datasets mais pas pu aller jusqu'à la procédure de fusion. 

Pour finir, on sauvegarde le dataset final au format CSV. 

In [39]:
#df=pd.read_csv("df4.csv")
#dfr=pd.read_csv("ratings4.csv")
res=pd.merge(df,dfr,on="nom")
res

,nom,pays,statut,genre,years,lyrical themes,note moyenne,favoris
0,Poison The Well,Etats-Unis,Split-Up,Post Hardcore,1997,NA,7.17,4
1,Tony MacAlpine,United States,Active,"Shred, Neoclassical, Progressive Rock/Metal",1985-,Instrumental,7.47,4
2,Gallhammer,Japan,On hold,"Black/Doom Metal, Crust",2003-2013,"Delirium, Paranoia, Hatred, Despair",7.43,7
3,Underoath,United States,Active,"Metalcore (early), Alternative Rock/Post-Hardc...","1997-2013, 2015-","Christianity (early); Consequence, Heartbreak,...",6.58,15
4,"Drop Dead, Gorgeous",Etats-Unis,Split-Up,Metalcore,2004,NA,0.00,0
5,Grand Magus,Sweden,Active,Heavy/Doom Metal,"1996-1999 , 1999-","Paganism, Anti-Christianity, Strength, Death",6.92,12
6,Possessed,United States,Active,Death/Thrash Metal,"1983-1987, 1990-1993, 2007-","Satanism, Occultism, Anti-Christianity, Murder...",7.15,7
7,Wolfmother,Australie,Actif,Stoner,2000,NA,7.00,69
8,Teräsbetoni,Finland,Active,Power Metal,"2003-2011, 2013, 2014, 2015-","Metal, Glory, Power, Justice, Honour, Battles",0.00,3
9,Revocation,United States,Active,Technical Death/Thrash Metal,"2000-2006 , 2006-","Death, Social criticism, Chaos, Pestilence",7.22,17


In [40]:
res.to_csv("final18.csv",index=False)